# Kundt’s Tube Speed of Sound Notebook (with Missing Modes)
This notebook demonstrates a **grid‐search** approach to handle missing harmonics (skipped modes),
so we do not force consecutive integer \(n\) assignments for each observed frequency.

## How It Works
1. We **sort** the observed resonance frequencies.
2. We **grid‐search** a range of possible fundamental frequencies \(f_1\) (from a minimum to a maximum around your lowest frequency).
3. For each candidate fundamental frequency \(f_{\mathrm{candidate}}\):
   - For each observed frequency \(f_i\), assign \(n_i = \mathrm{round}(f_i / f_{\mathrm{candidate}})\). If that's <1, force it to 1.
   - Compute a forced‐zero‐intercept slope \(\hat{m}\) from these \((n_i, f_i)\) pairs.
   - Calculate the SSE = \(\sum (f_i - \hat{m} n_i)^2\).
4. Choose the \(f_{\mathrm{candidate}}\) that gives the **lowest SSE**.
5. Re‐assign the final \(n_i\) and re‐compute slope, speed of sound, \(R^2\), etc.

This approach is more robust when you know you have **missing** integer modes.


In [ ]:
# ---------------------------------------------------------
# User Inputs
# ---------------------------------------------------------

# 1) Meter-stick readings (in centimeters)
#    Example: Tube from 12.3 cm to 46.3 cm -> 34 cm length
length_start_cm = 12.3
length_end_cm   = 46.3

# 2) Observed resonance frequencies (in any order). Possibly missing some modes.
observed_freqs = [5050, 723, 1449, 3608, 2890, 4403, 2165]

# 3) Plot title
plot_title = "Kundt’s Tube Resonances: Observed vs. Regression (Grid Search)"

# 4) Grid-search resolution
#    We'll define how finely we sample possible fundamental frequencies.
#    If your frequencies are in the thousands, a step of e.g. 0.5 or 1.0 might be enough.
f1_step = 1.0

print("User inputs have been set. Proceed to run the next cell.")

In [ ]:
# ---------------------------------------------------------
# Main Analysis and Plotting with Grid Search
# ---------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt

def forced_zero_intercept_linear_regression(x_vals, y_vals):
    """
    Returns slope 'm' from a forced-zero-intercept fit y = m*x.
    Formula: m = sum(x*y) / sum(x^2).
    """
    x_arr = np.array(x_vals, dtype=float)
    y_arr = np.array(y_vals, dtype=float)
    m = np.sum(x_arr * y_arr) / np.sum(x_arr * x_arr)
    return m

def compute_r_squared_forced_intercept(x_vals, y_vals, slope):
    """
    Computes R^2 for a forced-zero intercept regression.
    We'll still use the standard R^2 definition:
        R^2 = 1 - SSE/SST,
    with SSE = sum((y_i - m*x_i)^2),
         SST = sum((y_i - mean(y))^2).
    """
    x_arr = np.array(x_vals, dtype=float)
    y_arr = np.array(y_vals, dtype=float)
    y_pred = slope * x_arr
    sse = np.sum((y_arr - y_pred)**2)
    sst = np.sum((y_arr - np.mean(y_arr))**2)
    if sst == 0:
        return 1.0 if sse == 0 else 0.0
    return 1.0 - (sse / sst)

# ---------------------------------------------------------
# 1) Compute tube length in meters.
# ---------------------------------------------------------
L_cm = length_end_cm - length_start_cm
L_m  = L_cm / 100.0  # convert cm -> m

# ---------------------------------------------------------
# 2) Sort the observed frequencies.
# ---------------------------------------------------------
observed_freqs.sort()
f_data = np.array(observed_freqs, dtype=float)
k = len(f_data)

# We'll define a search range for the fundamental frequency.
# Lower bound: maybe ~ f_data[0]/(k+1), times some factor
# Upper bound: maybe f_data[0]*1.1 to allow it to be slightly bigger.
# The exact choice is somewhat arbitrary, but works in practice.

f1_min = 0.5 * f_data[0] / (k+1)  # half of that ratio
f1_max = 1.1 * f_data[0]

if f1_min < 1.0:
    f1_min = 1.0  # we won't search below 1 Hz to avoid nonsense

f1_candidates = np.arange(f1_min, f1_max, f1_step)
if len(f1_candidates) == 0:
    # fallback if the range is too narrow
    f1_candidates = np.array([f_data[0]/(k+1), f_data[0]])

best_sse = None
best_f1  = None
best_n   = None

# ---------------------------------------------------------
# 3) Grid search over possible fundamental frequencies.
# ---------------------------------------------------------

for f1_cand in f1_candidates:
    # Assign integer modes for each frequency
    n_cand = []
    for f in f_data:
        n_i = int(round(f / f1_cand))
        if n_i < 1:
            n_i = 1
        n_cand.append(n_i)

    n_cand = np.array(n_cand, dtype=float)

    # Forced-intercept slope for this assignment
    slope_cand = forced_zero_intercept_linear_regression(n_cand, f_data)

    # SSE
    y_pred = slope_cand * n_cand
    sse_cand = np.sum((f_data - y_pred)**2)

    if (best_sse is None) or (sse_cand < best_sse):
        best_sse = sse_cand
        best_f1  = f1_cand
        best_n   = n_cand

# ---------------------------------------------------------
# 4) Use the best f1 to finalize the assignment.
# ---------------------------------------------------------
n_final = best_n
slope_final = forced_zero_intercept_linear_regression(n_final, f_data)
f_calc = slope_final * n_final

# Speed of sound: slope_final = v/(2L) => v = slope_final * 2L
speed_of_sound = slope_final * 2.0 * L_m

# R^2
r2_final = compute_r_squared_forced_intercept(n_final, f_data, slope_final)

# Sort results by assigned n
sort_idx = np.argsort(n_final)
n_int_sorted = n_final[sort_idx].astype(int)
f_obs_sorted = f_data[sort_idx]
f_calc_sorted = f_calc[sort_idx]

# Identify missing n in [min(n_int_sorted), max(n_int_sorted)]
all_n_range = np.arange(n_int_sorted.min(), n_int_sorted.max()+1)
missing_n = [ni for ni in all_n_range if ni not in n_int_sorted]

# Fundamental freq from slope:
fundamental_freq_est = slope_final  # because f(n)=slope_final*n, so at n=1 => slope_final

# ---------------------------------------------------------
# 5) Print out the table.
# ---------------------------------------------------------
print("Assigned Harmonic Numbers (with grid-search approach):")
print("  n   f_obs [Hz]   f_calc [Hz]")
for i in range(len(n_int_sorted)):
    print(f"{n_int_sorted[i]:3d}   {f_obs_sorted[i]:9.2f}   {f_calc_sorted[i]:10.2f}")

if missing_n:
    print("\nMissing modes (no measured freq):", missing_n)
else:
    print("\nNo missing integer modes in this sequence.")

print(f"\nBest SSE from grid search: {best_sse:.2f}")
print(f"Extracted slope (final regression) = {slope_final:.3f} Hz/mode")
print(f"Tube length = {L_cm:.2f} cm => {L_m:.3f} m")
print(f"Estimated fundamental frequency f1 ≈ {fundamental_freq_est:.2f} Hz")
print(f"Speed of sound = {speed_of_sound:.2f} m/s")
print(f"R^2 = {r2_final:.4f}")

# ---------------------------------------------------------
# 6) Plot
# ---------------------------------------------------------
plt.figure(figsize=(7,5))
# Plot observed data
plt.scatter(n_int_sorted, f_obs_sorted, color='blue', label='Observed frequencies')

# Best-fit line from forced intercept
n_plot = np.linspace(n_int_sorted.min(), n_int_sorted.max(), 200)
f_plot = slope_final * n_plot
plt.plot(n_plot, f_plot, color='red', label='Regression line')

# Only plot missing n in orange
if missing_n:
    missing_n_arr = np.array(missing_n, dtype=float)
    f_missing_calc = slope_final * missing_n_arr
    plt.scatter(missing_n_arr, f_missing_calc, color='orange', marker='x',
                label='Calculated for missing modes')

plt.xlabel('Harmonic number n')
plt.ylabel('Frequency [Hz]')
plt.title(plot_title)
plt.grid(True)
plt.legend(loc='upper right')

# Annotate text
equation_str = f"f(n) = {slope_final:.2f}·n"
text_str = (f"{equation_str}\n"
            f"R² = {r2_final:.3f}\n"
            f"Tube length L = {L_m:.3f} m\n"
            f"Fundamental f₁ ≈ {fundamental_freq_est:.1f} Hz\n"
            f"Speed of sound v ≈ {speed_of_sound:.1f} m/s")

plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes,
         fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

plt.tight_layout()

png_filename = "kundts_tube_resonances_gridsearch.png"
plt.savefig(png_filename, dpi=300)
print(f"\nFigure saved as {png_filename}")

plt.show()
